### Initialization

In [ ]:
import eikon as ek
import refinitiv.data as rd
import pandas as pd
import os
import sys
import datetime as dt

from retrying import retry
from dotenv import load_dotenv
from tqdm import tqdm
from bs4 import BeautifulSoup

sys.path.insert(0, r'c:\Users\joneh\master_thesis\src')
from main_utils import *
from db_utils import *

load_dotenv()
ek.set_app_key(os.getenv('EIKON_API_KEY'))
rd.open_session(app_key=os.getenv('EIKON_API_KEY'))

## Resources

add link here
[EIKON information sheet](https://developers.lseg.com/content/dam/devportal/api-families/eikon/eikon-data-api/documentation/eikon_data_api_for_python_v1.pdf)

### Parameters

#### Topics

- **Crude Oil**: Topic:CRU

- **Oil Storage**: Topic:OSTR

| Topics   |      Code      |  Cool |
|----------|:-------------:|------:|
| Crude Oil |  CRU | $1600 |
| Oil Storage |    OSTR   |   $12 |
| col 3 is | right-aligned |    $1 |

In [ ]:
response = rd.content.news.headlines.Definition(
        query=query, 
        extended_params={"searchIn": "FullStory"}, 
        count=1000, date_from='2023-01-03', date_to='2023-01-03'
    ).get_data()

response.data.df

In [ ]:
TOPICS = 'CRU OR OSTR'
query = f'Topic:{TOPICS} AND language:LEN'

start_date = dt.datetime(2023, 1, 1, 0, 0, 0)
end_date = dt.datetime(2023, 12, 31, 23, 59, 59)
dates = pd.date_range(start_date, end_date, freq='D')

df_list = []

for date in tqdm(dates):
    response = rd.content.news.headlines.Definition(
        query=query, 
        extended_params={"searchIn": "FullStory"}, 
        count=1000, date_from=date, date_to=date
    ).get_data()

    df_list.append(response.data.df)

df = pd.concat(df_list)

display(df)

In [ ]:
# Enter filename here:
file_name = f'CRU_2022.csv'
# Enter relative path for saving the file:
relative_path = 'data/news'

df.to_csv(save_path(relative_path, file_name), index=True)